# Mnist DataScience (Training Stage)

## Develop training workflow

  1. develop component within the workflow (requirement.txt, script.py)
  3. define workflow (dependencies, parameters)

## Deploy training workflow
  
  1. choose deploying backend
  2. build/run workflow to train checker
  3. build/run workflow to train model
  
## Download the final prepared model/Push the model(future)

  1. download checker model (e.g., drift_detector)
  2. download model (e.g., mnist_cnn)
  
## Clean training environment
  
  1. clean training workflow, delete namespace
  
## Submit all model and artifacts to Scanflow Server
```bash
  mnist
    - workflow
       - data
         - mnist
           - train_images.npy
           - train_labels.npy
           - test_images.npy
           - test_labels.npy
       - modeling
         - modeling.py
         - req_modeling.txt
       - checker
         - checker.py
         - req_checker.txt
    - meta
       - mnist-wf.json
       - mnist-checker-wf.json
       - Dockerfile-modeling
       - Dockerfile-checker
       - mnist_cnn(model)
       - drift_detector(model)
```

## Demo: Train workflows

In [3]:
import sys
import matplotlib.pyplot as plt
import os
sys.path.insert(0,'../..')

import scanflow

from scanflow.setup import Setup, Executor, Workflow
from scanflow.special import Tracker, Checker, Improver, Planner
from scanflow.client import ScanflowClient

# App folder
base_path = os.path.dirname(os.getcwd())
app_dir = os.path.join(base_path, "examples/mnist-DS/workflow")
app_name = "mnist"


executors = [Executor(name='modeling-mnist', 
                      file='modeling.py',
                      parameters={'x_train_path': './data/mnist/train_images.npy', 
                                  'y_train_path': './data/mnist/train_labels.npy',
                                  'x_test_path': './data/mnist/test_images.npy',
                                  'y_test_path': './data/mnist/test_labels.npy'},
                      requirements='req_modeling.txt')           
            ]


workflow1 = Workflow(name='mnist', 
                     executors=executors,
                     tracker=Tracker)
    
setup = Setup(app_name, app_dir, workflows=[workflow1],
                             verbose=True)

In [5]:
# scanflow client
client = ScanflowClient(server_uri="http://172.30.0.50:46666",
                        server_tracker_uri="http://172.30.0.50:46667",
                        verbose=True)

In [7]:
# deploy backend
argo = client.get_deploy_backend(setup, 
                  backend="argo", 
                  namespace="mnist-training",
                  registry="172.30.0.49:5000",
                  k8sconfigdir="/gpfs/bsc_home/xpliu/.kube/config")

15-Apr-21 19:32:00 -  INFO - [++]Creating namespace "mnist-training"
15-Apr-21 19:32:00 -  INFO - [++]Creating Role for 'default service account'


In [ ]:
argo.build_workflows()

## Submit All